In [ ]:
#!/usr/bin/env python3
"""
dist_fit_evaluation.py

Fit multiple parametric distributions to channel histograms of first 5 images in a folder.
Reports MLE log-likelihood, AIC, BIC, KS test, and 5-fold CV log-likelihood.
Selects best distribution by BIC (primary) and by CV log-likelihood (secondary).
"""

import os
import csv
import numpy as np
from PIL import Image
import torchvision.transforms as T
from scipy.stats import norm, laplace, t, logistic, cauchy, kstest
from sklearn.model_selection import KFold

# ----------------- CONFIG -----------------
IMAGE_FOLDER = "/dcs/large/u2157170/code/results/"  # change if needed
N_IMAGES = 15
EXTENSIONS = {".png", ".jpg", ".jpeg"}
N_BINS = 100
CV_FOLDS = 5
OUTPUT_CSV = "dist_fit_summary_first5.csv"
# ------------------------------------------

# transform: to tensor and pad as your previous pipeline
transform = T.Compose([
    T.ToTensor(),
    T.Pad((0, 0, 8, 8))  # match your prior padding
])

# Candidate distributions (GenGaussian removed)
DISTS = {
    "Gaussian": {
        "fit": lambda x: norm.fit(x),
        "logpdf": lambda x, p: norm.logpdf(x, loc=p[0], scale=p[1]),
        "k": 2
    },
    "Laplace": {
        "fit": lambda x: laplace.fit(x),
        "logpdf": lambda x, p: laplace.logpdf(x, loc=p[0], scale=p[1]),
        "k": 2
    },
    "StudentT": {
        "fit": lambda x: t.fit(x),
        "logpdf": lambda x, p: t.logpdf(x, p[0], loc=p[1], scale=p[2]),
        "k": 3
    },
    "Logistic": {
        "fit": lambda x: logistic.fit(x),
        "logpdf": lambda x, p: logistic.logpdf(x, loc=p[0], scale=p[1]),
        "k": 2
    },
    "Cauchy": {
        "fit": lambda x: cauchy.fit(x),
        "logpdf": lambda x, p: cauchy.logpdf(x, loc=p[0], scale=p[1]),
        "k": 2
    }
}

def get_first_images(folder, count=N_IMAGES, extensions=EXTENSIONS):
    files = [os.path.join(folder, f) for f in os.listdir(folder)
             if os.path.splitext(f)[1].lower() in extensions]
    return sorted(files)[:count]

def compute_metrics_for_latents(latents: np.ndarray):
    """
    Fit each distribution, compute MLE log-likelihood, AIC, BIC, KS stat & p-value,
    and CV (out-of-sample) log-likelihood using CV_FOLDS splits.
    Returns a dict of results keyed by distribution name.
    """
    results = {}
    n = latents.size
    if n < 2:
        return results

    kf = KFold(n_splits=min(CV_FOLDS, n), shuffle=True, random_state=0)

    for name, spec in DISTS.items():
        try:
            params = spec["fit"](latents)
            logpdf_vals = spec["logpdf"](latents, params)
            ll = float(np.sum(logpdf_vals))

            k = spec["k"]
            aic = 2 * k - 2 * ll
            bic = k * np.log(n) - 2 * ll

            if name == "Gaussian":
                cdf_func = lambda x: norm.cdf(x, loc=params[0], scale=params[1])
            elif name == "Laplace":
                cdf_func = lambda x: laplace.cdf(x, loc=params[0], scale=params[1])
            elif name == "StudentT":
                cdf_func = lambda x: t.cdf(x, params[0], loc=params[1], scale=params[2])
            elif name == "Logistic":
                cdf_func = lambda x: logistic.cdf(x, loc=params[0], scale=params[1])
            elif name == "Cauchy":
                cdf_func = lambda x: cauchy.cdf(x, loc=params[0], scale=params[1])
            else:
                cdf_func = None

            if cdf_func is not None:
                ks_stat, ks_p = kstest(latents, cdf_func)
            else:
                ks_stat, ks_p = np.nan, np.nan

            # CV: out-of-sample log-likelihood
            cv_lls = []
            for train_idx, test_idx in kf.split(latents):
                train = latents[train_idx]
                test = latents[test_idx]
                try:
                    p_train = spec["fit"](train)
                    ll_test = np.sum(spec["logpdf"](test, p_train))
                    cv_lls.append(ll_test)
                except Exception:
                    cv_lls.append(np.nan)
            cv_ll = float(np.nansum(cv_lls))
            cv_ll_mean = float(np.nanmean(cv_lls)) if len(cv_lls) > 0 else np.nan

            results[name] = {
                "params": params,
                "n": int(n),
                "ll": ll,
                "aic": float(aic),
                "bic": float(bic),
                "ks_stat": float(ks_stat),
                "ks_p": float(ks_p),
                "cv_ll_sum": float(cv_ll),
                "cv_ll_mean": float(cv_ll_mean),
                "k": int(k)
            }
        except Exception as e:
            results[name] = {"error": str(e)}
    return results

def evaluate_folder(folder):
    image_paths = get_first_images(folder, count=N_IMAGES, extensions=EXTENSIONS)
    rows = []
    for idx, path in enumerate(image_paths):
        try:
            img = Image.open(path).convert("RGB")
        except Exception as e:
            print(f"Skipping {path}: {e}")
            continue

        x = transform(img).squeeze(0).numpy()  # (C,H,W)
        print(f"\nImage {idx+1}: {os.path.basename(path)}  padded shape: {x.shape}")

        for ch, ch_name in enumerate(["R", "G", "B"]):
            latents = x[ch].ravel().astype(np.float64)
            latents = latents[np.isfinite(latents)]
            if latents.size == 0:
                continue

            res = compute_metrics_for_latents(latents)

            bic_best = None
            cv_best = None
            for name, r in res.items():
                if "error" in r:
                    continue
                if bic_best is None or r["bic"] < res[bic_best]["bic"]:
                    bic_best = name
                if cv_best is None or r["cv_ll_sum"] > res[cv_best]["cv_ll_sum"]:
                    cv_best = name

            print(f" Channel {ch_name}: samples={latents.size}")
            for name, r in res.items():
                if "error" in r:
                    print(f"  {name}: ERROR {r['error']}")
                    continue
                print(f"  {name:12s}  ll={r['ll']:.1f}  AIC={r['aic']:.1f}  BIC={r['bic']:.1f}  "
                      f"KS={r['ks_stat']:.3f}/{r['ks_p']:.3f}  CV_sum={r['cv_ll_sum']:.1f}")

            print(f"  -> Best (BIC): {bic_best}, Best (CV): {cv_best}")

            for name, r in res.items():
                row = {
                    "image": os.path.basename(path),
                    "channel": ch_name,
                    "distribution": name
                }
                if "error" in r:
                    row.update({"error": r["error"]})
                else:
                    row.update({
                        "n": r["n"],
                        "ll": r["ll"],
                        "aic": r["aic"],
                        "bic": r["bic"],
                        "ks_stat": r["ks_stat"],
                        "ks_p": r["ks_p"],
                        "cv_ll_sum": r["cv_ll_sum"],
                        "cv_ll_mean": r["cv_ll_mean"],
                        "k": r["k"]
                    })
                rows.append(row)

    fieldnames = ["image", "channel", "distribution", "n", "k", "ll", "aic", "bic",
                  "ks_stat", "ks_p", "cv_ll_sum", "cv_ll_mean", "error"]
    with open(OUTPUT_CSV, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            out = {k: r.get(k, "") for k in fieldnames}
            writer.writerow(out)

    print(f"\nSaved summary CSV to {OUTPUT_CSV}")

if __name__ == "__main__":
    evaluate_folder(IMAGE_FOLDER)



Image 1: S2A_MSIL2A_20170905T095031_N9999_R079_T35VNL_00_00_RGB.png  padded shape: (3, 128, 128)
 Channel R: samples=16384
  Gaussian      ll=3655.0  AIC=-7305.9  BIC=-7290.5  KS=0.117/0.000  CV_sum=3652.4
  Laplace       ll=3493.0  AIC=-6982.1  BIC=-6966.7  KS=0.127/0.000  CV_sum=3489.4
  StudentT      ll=3661.8  AIC=-7317.6  BIC=-7294.5  KS=0.107/0.000  CV_sum=3654.4
  Logistic      ll=3641.7  AIC=-7279.5  BIC=-7264.1  KS=0.102/0.000  CV_sum=3637.9
  Cauchy        ll=1875.8  AIC=-3747.6  BIC=-3732.2  KS=0.148/0.000  CV_sum=1871.2
  -> Best (BIC): StudentT, Best (CV): StudentT
 Channel G: samples=16384
  Gaussian      ll=2331.2  AIC=-4658.4  BIC=-4643.0  KS=0.079/0.000  CV_sum=2329.8
  Laplace       ll=1797.1  AIC=-3590.2  BIC=-3574.8  KS=0.067/0.000  CV_sum=1793.3
  StudentT      ll=2331.2  AIC=-4656.4  BIC=-4633.3  KS=0.079/0.000  CV_sum=2329.8
  Logistic      ll=2100.3  AIC=-4196.6  BIC=-4181.1  KS=0.073/0.000  CV_sum=2098.4
  Cauchy        ll=-309.6  AIC=623.1  BIC=638.6  KS=0.10

In [5]:
#!/usr/bin/env python3
"""
dist_fit_evaluation.py

Fit Gaussian, Laplace, and Student's t distributions to channel histograms of first N images.
Reports MLE log-likelihood, AIC, BIC, KS test, and 5-fold CV log-likelihood.
Selects best distribution by BIC (primary) and prints a summary count at the end.
"""

import os
import csv
import numpy as np
from PIL import Image
import torchvision.transforms as T
from scipy.stats import norm, laplace, t, kstest
from sklearn.model_selection import KFold

# ----------------- CONFIG -----------------
IMAGE_FOLDER = "/dcs/large/u2157170/code/results/"  # change if needed
N_IMAGES = 5
EXTENSIONS = {".png", ".jpg", ".jpeg"}
CV_FOLDS = 5
OUTPUT_CSV = "dist_fit_summary_first5.csv"
# ------------------------------------------

transform = T.Compose([
    T.ToTensor(),
    T.Pad((0, 0, 8, 8))  # match your prior padding
])

# Candidate distributions (only 3 now)
DISTS = {
    "Gaussian": {
        "fit": lambda x: norm.fit(x),
        "logpdf": lambda x, p: norm.logpdf(x, loc=p[0], scale=p[1]),
        "cdf": lambda x, p: norm.cdf(x, loc=p[0], scale=p[1]),
        "k": 2
    },
    "Laplace": {
        "fit": lambda x: laplace.fit(x),
        "logpdf": lambda x, p: laplace.logpdf(x, loc=p[0], scale=p[1]),
        "cdf": lambda x, p: laplace.cdf(x, loc=p[0], scale=p[1]),
        "k": 2
    },
    "StudentT": {
        "fit": lambda x: t.fit(x),
        "logpdf": lambda x, p: t.logpdf(x, p[0], loc=p[1], scale=p[2]),
        "cdf": lambda x, p: t.cdf(x, p[0], loc=p[1], scale=p[2]),
        "k": 3
    }
}

def get_first_images(folder, count=N_IMAGES, extensions=EXTENSIONS):
    files = [os.path.join(folder, f) for f in os.listdir(folder)
             if os.path.splitext(f)[1].lower() in extensions]
    return sorted(files)[:count]

def compute_metrics_for_latents(latents: np.ndarray):
    results = {}
    n = latents.size
    if n < 2:
        return results

    kf = KFold(n_splits=min(CV_FOLDS, n), shuffle=True, random_state=0)

    for name, spec in DISTS.items():
        try:
            params = spec["fit"](latents)
            logpdf_vals = spec["logpdf"](latents, params)
            ll = float(np.sum(logpdf_vals))

            k = spec["k"]
            aic = 2 * k - 2 * ll
            bic = k * np.log(n) - 2 * ll

            ks_stat, ks_p = kstest(latents, lambda x: spec["cdf"](x, params))

            cv_lls = []
            for train_idx, test_idx in kf.split(latents):
                train = latents[train_idx]
                test = latents[test_idx]
                try:
                    p_train = spec["fit"](train)
                    ll_test = np.sum(spec["logpdf"](test, p_train))
                    cv_lls.append(ll_test)
                except Exception:
                    cv_lls.append(np.nan)
            cv_ll = float(np.nansum(cv_lls))
            cv_ll_mean = float(np.nanmean(cv_lls)) if len(cv_lls) > 0 else np.nan

            results[name] = {
                "params": params,
                "n": int(n),
                "ll": ll,
                "aic": float(aic),
                "bic": float(bic),
                "ks_stat": float(ks_stat),
                "ks_p": float(ks_p),
                "cv_ll_sum": float(cv_ll),
                "cv_ll_mean": float(cv_ll_mean),
                "k": int(k)
            }
        except Exception as e:
            results[name] = {"error": str(e)}
    return results

def evaluate_folder(folder):
    image_paths = get_first_images(folder, count=N_IMAGES, extensions=EXTENSIONS)
    rows = []
    summary_counts = {"Gaussian": 0, "Laplace": 0, "StudentT": 0}

    for idx, path in enumerate(image_paths):
        try:
            img = Image.open(path).convert("RGB")
        except Exception as e:
            print(f"Skipping {path}: {e}")
            continue

        x = transform(img).squeeze(0).numpy()
        print(f"\nImage {idx+1}: {os.path.basename(path)}  padded shape: {x.shape}")

        for ch, ch_name in enumerate(["R", "G", "B"]):
            latents = x[ch].ravel().astype(np.float64)
            latents = latents[np.isfinite(latents)]
            if latents.size == 0:
                continue

            res = compute_metrics_for_latents(latents)

            bic_best = None
            cv_best = None
            for name, r in res.items():
                if "error" in r:
                    continue
                if bic_best is None or r["bic"] < res[bic_best]["bic"]:
                    bic_best = name
                if cv_best is None or r["cv_ll_sum"] > res[cv_best]["cv_ll_sum"]:
                    cv_best = name

            if bic_best:
                summary_counts[bic_best] += 1

            print(f" Channel {ch_name}: samples={latents.size}")
            for name, r in res.items():
                if "error" in r:
                    print(f"  {name}: ERROR {r['error']}")
                    continue
                print(f"  {name:10s} ll={r['ll']:.1f}  AIC={r['aic']:.1f}  BIC={r['bic']:.1f}  "
                      f"KS={r['ks_stat']:.3f}/{r['ks_p']:.3f}  CV_sum={r['cv_ll_sum']:.1f}")

            print(f"  -> Best (BIC): {bic_best}, Best (CV): {cv_best}")

            for name, r in res.items():
                row = {"image": os.path.basename(path), "channel": ch_name, "distribution": name}
                if "error" in r:
                    row.update({"error": r["error"]})
                else:
                    row.update({
                        "n": r["n"], "ll": r["ll"], "aic": r["aic"], "bic": r["bic"],
                        "ks_stat": r["ks_stat"], "ks_p": r["ks_p"],
                        "cv_ll_sum": r["cv_ll_sum"], "cv_ll_mean": r["cv_ll_mean"], "k": r["k"]
                    })
                rows.append(row)

    fieldnames = ["image", "channel", "distribution", "n", "k", "ll", "aic", "bic",
                  "ks_stat", "ks_p", "cv_ll_sum", "cv_ll_mean", "error"]
    with open(OUTPUT_CSV, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            writer.writerow({k: r.get(k, "") for k in fieldnames})

    print(f"\nSaved summary CSV to {OUTPUT_CSV}")
    print("\n=== Overall Summary (Best by BIC) ===")
    for dist, count in summary_counts.items():
        print(f"  {dist:10s}: {count} channels selected")

if __name__ == "__main__":
    evaluate_folder(IMAGE_FOLDER)



Image 1: S2A_MSIL2A_20170905T095031_N9999_R079_T35VNL_00_00_RGB.png  padded shape: (3, 128, 128)
 Channel R: samples=16384
  Gaussian   ll=3655.0  AIC=-7305.9  BIC=-7290.5  KS=0.117/0.000  CV_sum=3652.4
  Laplace    ll=3493.0  AIC=-6982.1  BIC=-6966.7  KS=0.127/0.000  CV_sum=3489.4
  StudentT   ll=3661.8  AIC=-7317.6  BIC=-7294.5  KS=0.107/0.000  CV_sum=3654.4
  -> Best (BIC): StudentT, Best (CV): StudentT
 Channel G: samples=16384
  Gaussian   ll=2331.2  AIC=-4658.4  BIC=-4643.0  KS=0.079/0.000  CV_sum=2329.8
  Laplace    ll=1797.1  AIC=-3590.2  BIC=-3574.8  KS=0.067/0.000  CV_sum=1793.3
  StudentT   ll=2331.2  AIC=-4656.4  BIC=-4633.3  KS=0.079/0.000  CV_sum=2329.8
  -> Best (BIC): Gaussian, Best (CV): Gaussian
 Channel B: samples=16384
  Gaussian   ll=4249.2  AIC=-8494.5  BIC=-8479.1  KS=0.081/0.000  CV_sum=4248.4
  Laplace    ll=4040.7  AIC=-8077.3  BIC=-8061.9  KS=0.065/0.000  CV_sum=4036.9
  StudentT   ll=4249.2  AIC=-8492.5  BIC=-8469.4  KS=0.081/0.000  CV_sum=4248.4
  -> Best 